In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from statsmodels.graphics.gofplots import qqplot
from numpy.linalg import inv
from joblib import dump, load
from sklearn.impute import SimpleImputer

/var/folders/t_/0sss_n354p10j1d1j41c4ml00000gn/T/ipykernel_33833/2721594553.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# *1. Baseline Model*

### Retrieve Data

In [2]:
df_baseline = pd.read_csv('./data/processed_scraped_apartment_sales_baseline.csv')
df = pd.read_csv('./data/processed_scraped_apartment_sales_processed.csv')

df_baseline = df_baseline.drop('Unnamed: 0', axis=1)
df = df.drop('Unnamed: 0', axis=1)
#df = df.drop(['broker', 'brokerage_firm', 'region', 'floor_number'], axis=1)
df = df.drop(['broker', 'brokerage_firm'], axis=1)


df_baseline.head()

,number_of_rooms,area_size,has_elevator,year_built,annual_fee_sek,annual_cost_sek,region_processed_bromma,region_processed_centrala sundbyberg,region_processed_gröndal,region_processed_hammarby sjöstad,...,region_processed_östermalm,has_balcony_nej,has_balcony_unknown,cleaned_floor_number,brokerage_firm_processed_erik olsson fastighetsförmedling,brokerage_firm_processed_fastighetsbyrån stockholm - södermalm,brokerage_firm_processed_länsförsäkringar fastighetsförmedling solna,brokerage_firm_processed_notar,brokerage_firm_processed_tradition mäkleri,price_sold_sek
0,3.0,74,False,1953.0,4899,9000,False,False,False,False,...,False,False,False,2.0,False,False,False,False,False,4400000
1,1.0,25,True,1961.0,2265,3120,False,False,False,False,...,False,False,True,4.0,False,False,False,False,False,1015000
2,1.0,40,True,1955.0,3250,4500,False,False,False,False,...,False,False,False,2.0,False,False,False,False,False,1395000
3,1.0,31,True,1963.0,1290,6804,False,False,False,False,...,False,False,False,1.0,False,False,False,True,False,3150000
4,1.0,40,True,1972.0,2947,4200,False,False,False,False,...,False,False,False,1.0,False,False,False,False,False,1750000


In [3]:
X = df_baseline.drop('price_sold_sek', axis=1)
y = df_baseline['price_sold_sek']

### Split Data

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

### Load Baseline Model and Predict

In [5]:
baseline_model = load('../src/models/baseline_model.joblib')

In [6]:
baseline_predictions = baseline_model.predict(X_test)

In [7]:
def evaluate_model(observed_values, predictions):
    mse = mean_squared_error(observed_values, predictions)
    rmse = mean_squared_error(observed_values, predictions, squared=False)
    mae = mean_absolute_error(observed_values, predictions)
    r2 = r2_score(observed_values, predictions)

    print(f"MSE: {mse} \nRMSE: {rmse} \nMAE: {mae} \nR-squared: {r2}")

In [8]:
print('Baseline results:')
evaluate_model(y_test, baseline_predictions)

Baseline results:
MSE: 3404845047266.328 
RMSE: 1845222.2216487445 
MAE: 1363035.2040760939 
R-squared: 0.5441301212408783


/Users/ossianhempel/repos/apartment_price_predictor/venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# *2. Impute Missing Data*

In [9]:
df.columns

Index(['region', 'price_sold_sek', 'number_of_rooms', 'area_size',
       'floor_number', 'has_elevator', 'year_built', 'annual_fee_sek',
       'annual_cost_sek', 'region_processed_bromma',
       'region_processed_centrala sundbyberg', 'region_processed_gröndal',
       'region_processed_hammarby sjöstad', 'region_processed_kungsholmen',
       'region_processed_råsunda', 'region_processed_södermalm',
       'region_processed_vasastan', 'region_processed_årsta',
       'region_processed_östermalm', 'has_balcony_nej', 'has_balcony_unknown',
       'cleaned_floor_number',
       'brokerage_firm_processed_erik olsson fastighetsförmedling',
       'brokerage_firm_processed_fastighetsbyrån stockholm - södermalm',
       'brokerage_firm_processed_länsförsäkringar fastighetsförmedling solna',
       'brokerage_firm_processed_notar',
       'brokerage_firm_processed_tradition mäkleri'],
      dtype='object')

In [10]:
df.isna().sum()

region                                                                    0
price_sold_sek                                                            0
number_of_rooms                                                           0
area_size                                                                 0
floor_number                                                              0
has_elevator                                                              0
year_built                                                                0
annual_fee_sek                                                            0
annual_cost_sek                                                           0
region_processed_bromma                                                   0
region_processed_centrala sundbyberg                                      0
region_processed_gröndal                                                  0
region_processed_hammarby sjöstad                                         0
region_proce

In [11]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

class PredictionBasedImputer:
    """
    A class for imputing missing values in a specified column of a DataFrame using predictive modeling.
    This version is adjusted to only consider numerical and binary variables as features for the regression model.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    target_col (str): The name of the column in the DataFrame for which missing values need to be imputed.
    """
    def __init__(self, df, target_col):
        self.df = df.copy()
        self.target_col = target_col
        self.model = LinearRegression()

    def filter_numerical_binary_columns(self, df):
        """
        Filters the DataFrame to include only numerical and binary columns.
        """
        numerical_binary_cols = df.select_dtypes(include=['number']).columns.tolist()
        # Optionally, check if binary columns are strictly 0/1 and include them.
        # For simplicity, this version considers all numerical columns.
        return df[numerical_binary_cols]

    def train_model(self):
        if self.df[self.target_col].isna().sum() == 0:
            raise ValueError(f"No missing values found in target column '{self.target_col}'.")

        # Filter dataframe for numerical and binary columns
        df_filtered = self.filter_numerical_binary_columns(self.df)

        # Separate the dataset into rows with and without missing target values
        df_with_target = df_filtered[df_filtered[self.target_col].notna()]
        df_missing_target = df_filtered[df_filtered[self.target_col].isna()]

        X = df_with_target.drop(columns=[self.target_col])
        y = df_with_target[self.target_col]

        if X.empty:
            raise ValueError("No numerical or binary data available to train the model.")

        X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model.fit(X_train, y_train)

        return df_missing_target.drop(columns=[self.target_col])

    def impute(self):
        df_missing_target = self.train_model()

        if df_missing_target.empty:
            print("No rows with missing target values to impute.")
            return self.df

        predicted_values = self.model.predict(df_missing_target)
        self.df.loc[self.df[self.target_col].isna(), self.target_col] = predicted_values

        return self.df

# Example usage
imputer = PredictionBasedImputer(df, 'cleaned_floor_number')
df = imputer.impute()


# *3. Process Outliers*

In [12]:
def quantile_based_flooring_and_outlier_tracking(df_or_series, lwr=5, upr=95):
    """
    Apply quantile based flooring to outliers in a DataFrame or Series.
    Outliers are identified using the IQR method and replaced using quantile flooring.
    Skips boolean and non-numeric columns. Tracks and prints the changes made.

    Parameters:
    df_or_series (pd.DataFrame or pd.Series): DataFrame or Series to process.
    lwr (int): Lower percentile for flooring.
    upr (int): Upper percentile for flooring.
    """
    def quantile_based_flooring(series, lwr, upr):
        if series.empty:
            return series  # Return empty series as is

        tenth_percentile = np.percentile(series, lwr)
        ninetieth_percentile = np.percentile(series, upr)
        output = np.where(series < tenth_percentile, tenth_percentile, series)
        output = np.where(output > ninetieth_percentile, ninetieth_percentile, output)
        return output

    changes = {}  # Dictionary to track changes

    if isinstance(df_or_series, pd.Series):
        df_or_series = pd.DataFrame(df_or_series)

    for column in df_or_series.columns:
        # Skip non-numeric and boolean columns
        if pd.api.types.is_numeric_dtype(df_or_series[column]) and not pd.api.types.is_bool_dtype(df_or_series[column]):
            series = df_or_series[column].copy()

            # Calculate Q1 (25th percentile) and Q3 (75th percentile)
            Q1 = series.quantile(0.25)
            Q3 = series.quantile(0.75)
            IQR = Q3 - Q1

            # Define bounds for outliers
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Identify outliers
            outliers = series[(series < lower_bound) | (series > upper_bound)]
            
            # Skip if no outliers are found
            if outliers.empty:
                continue

            flooring_values = quantile_based_flooring(outliers, lwr, upr)

            # Track changes
            for idx, val in zip(outliers.index, flooring_values):
                if series.loc[idx] != val:
                    changes.setdefault(column, []).append((idx, series.loc[idx], val))
                    series.loc[idx] = val

            # Update the DataFrame or Series
            df_or_series[column] = series

    # Display changes
    for column, column_changes in changes.items():
        print(f"Column: {column}")
        for change in column_changes:
            print(f"Index: {change[0]}, Original: {change[1]}, New: {change[2]}")

# Example usage
quantile_based_flooring_and_outlier_tracking(df)

Column: price_sold_sek
Index: 103, Original: 23700000, New: 20000000.0
Index: 427, Original: 21900000, New: 20000000.0
Index: 473, Original: 9200000, New: 9300000.0
Index: 923, Original: 9200000, New: 9300000.0
Index: 1094, Original: 23300000, New: 20000000.0
Index: 1223, Original: 25600000, New: 20000000.0
Index: 1553, Original: 9250000, New: 9300000.0
Index: 1912, Original: 9225000, New: 9300000.0
Index: 2069, Original: 26500000, New: 20000000.0
Column: area_size
Index: 73, Original: 1523, New: 1032.5
Index: 104, Original: 215.0, New: 287.0
Index: 107, Original: 215.0, New: 287.0
Index: 215, Original: 1191.0, New: 1032.5
Index: 275, Original: 1045.0, New: 1032.5
Index: 427, Original: 180.0, New: 287.0
Index: 465, Original: 1065.0, New: 1032.5
Index: 514, Original: 1085.0, New: 1032.5
Index: 542, Original: 180.0, New: 287.0
Index: 640, Original: 1388.0, New: 1032.5
Index: 703, Original: 1095.0, New: 1032.5
Index: 708, Original: 1165.0, New: 1032.5
Index: 726, Original: 1283.0, New: 10

/var/folders/t_/0sss_n354p10j1d1j41c4ml00000gn/T/ipykernel_33833/302766053.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1032.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  series.loc[idx] = val
/var/folders/t_/0sss_n354p10j1d1j41c4ml00000gn/T/ipykernel_33833/302766053.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7868.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  series.loc[idx] = val
/var/folders/t_/0sss_n354p10j1d1j41c4ml00000gn/T/ipykernel_33833/302766053.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '40286.399999999994' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  series.loc[idx] = va

### Train Model

In [13]:
X = df.drop(['price_sold_sek', 'region', 'floor_number'], axis=1)
y = df[['price_sold_sek']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [14]:
model = LinearRegression()

# Train your model with the transformed y_train
model.fit(X_train, y_train)

# Make predictions (these predictions will be in the log scale)
predictions = model.predict(X_test)

In [15]:
print('Results after dealing with missing values and outliers')
evaluate_model(y_test, predictions)

Results after dealing with missing values and outliers
MSE: 3131825588664.328 
RMSE: 1769696.4679470682 
MAE: 1308853.4753398541 
R-squared: 0.5104585245216431


/Users/ossianhempel/repos/apartment_price_predictor/venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# *4. Data Transformations*

### Log Transformation of Dependent Variables

In [16]:
# Perform Log transformation
y_train_log = np.log1p(y_train)

#### Train Model

In [17]:
model = LinearRegression()

# Train your model with the transformed y_train
model.fit(X_train, y_train_log)

# Make predictions (these predictions will be in the log scale)
log_predictions = model.predict(X_test)

# Transform predictions back to original scale
predictions = np.expm1(log_predictions)

In [18]:
print('Log transformation results:')
evaluate_model(y_test, predictions)

Log transformation results:
MSE: 2835329270104.6895 
RMSE: 1683843.6002505368 
MAE: 1155020.7282621097 
R-squared: 0.5568044148505771


/Users/ossianhempel/repos/apartment_price_predictor/venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Log transformation had a slight performance improvement in all metrics

### Feature Scaling

In [19]:
# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train)

# Transform (scale) both the training and test data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Now, you can use the scaled data for model training and predictions
model.fit(X_train_scaled, y_train_log)

# Make predictions (these predictions will be in the log scale)
log_predictions = model.predict(X_test_scaled)  # Corrected to use X_test_scaled

# Transform predictions back to original scale
predictions = np.expm1(log_predictions)

#### Train Model

In [20]:
evaluate_model(y_test, predictions)

MSE: 2835329270104.6685 
RMSE: 1683843.6002505305 
MAE: 1155020.7282621083 
R-squared: 0.5568044148505804


/Users/ossianhempel/repos/apartment_price_predictor/venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# *6. Export Loaded Data*

In [21]:
df = df.drop(['region', 'floor_number'], axis=1)

In [22]:
df.to_csv('./data/loaded_scraped_apartment_sales_data.csv')